In [3]:
!conda info --envs


# conda environments:
#
base                   C:\Users\evang\anaconda3
DATA301                C:\Users\evang\anaconda3\envs\DATA301
Data301                C:\Users\evang\anaconda3\envs\Data301
VDCenv                 C:\Users\evang\anaconda3\envs\VDCenv
myenvLLM             * C:\Users\evang\anaconda3\envs\myenvLLM



In [2]:
from openai import OpenAI
import os

def identify_security_concerns(scenario, question="What could be the security concerns?", model="gpt-3.5-turbo"):
    """
    Function to identify security concerns based on a company scenario using OpenAI's GPT model.

    Parameters:
        scenario (str): The operational details or scenario of the company.
        question (str): The specific security-related question to ask the model.
        model (str): The OpenAI GPT model to use (default: "gpt-3.5-turbo").

    Returns:
        str: The model's response to the security question.
    """
    # Construct the messages for the model
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert specializing in identifying risks and vulnerabilities."},
        {"role": "user", "content": f"Scenario: {scenario}"},
        {"role": "user", "content": f"Question: {question}"}
    ]

    # Use OpenAI's API to get a response
    try:
        
        # client = OpenAI()
        # response = client.completions.create(
        #     model=model,
        #     messages=messages
        # )
        # return response['choices'][0]['message']['content'].strip()

        client = OpenAI()
        
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        
        print(completion.choices[0].message)

        
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
if __name__ == "__main__":
    # Replace with your OpenAI API key
    # openai.api_key = "sk-abcd1234efgh5678abcd1234efgh5678abcd1234"
    api_key = os.getenv('OPENAI_API_KEY')

    # Provide a company scenario
    scenario = """
    A mid-sized company operates in the financial sector. They handle sensitive customer data, including personal 
    and financial information. Employees frequently access cloud-based systems for processing data and use 
    third-party APIs for transactions. Remote work is common, with employees using personal devices to log in. 
    The company also uses IoT devices for office automation.
    """

    # Ask the security-related question
    security_concerns = identify_security_concerns(scenario)
    print("Security Concerns Identified:")
    print(security_concerns)


Security Concerns Identified:
An error occurred: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


In [8]:
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer

# Suppress warnings
warnings.filterwarnings("ignore")

def identify_security_concerns(scenario, attack_map, question="What are the security concerns based on the following scenario?", model_name="EleutherAI/gpt-neo-1.3B"):
    """
    Function to identify security concerns based on a company scenario and predefined attack types using a Hugging Face model.

    Parameters:
        scenario (str): The operational details or scenario of the company.
        attack_map (dict): A dictionary mapping attack types to their descriptions.
        question (str): The specific security-related question to ask the model.
        model_name (str): The Hugging Face model to use (default: "EleutherAI/gpt-neo-1.3B").

    Returns:
        str: The structured list of security concerns with reasons.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Create a detailed context incorporating the attack map
    attack_descriptions = "\n".join([f"{attack}: {description}" for attack, description in attack_map.items()])
    full_context = (
        f"Scenario: {scenario}\n"
        f"Here is a map of potential security attacks and their descriptions:\n{attack_descriptions}\n"
        f"Question: {question}\nPlease identify the relevant security concerns and explain why.\nAnswer:"
    )

    # Tokenize the input
    inputs = tokenizer(full_context, return_tensors="pt")

    # Generate the response
    outputs = model.generate(**inputs, max_new_tokens=200)
    raw_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print raw response for debugging
    print("Raw Response:", raw_response)

    # Extract and structure the relevant part of the answer
    if "Answer:" in raw_response:
        raw_response = raw_response.split("Answer:")[-1].strip()

    # Initialize a list to hold the results
    attack_results = []

    # Loop through each attack in the attack map and check if key phrases are mentioned in the response or scenario
    for attack, description in attack_map.items():
        # Check if any key phrases are found in the raw response or scenario
        key_phrases = [attack.lower(), description.lower()]
        if any(phrase in raw_response.lower() or phrase in scenario.lower() for phrase in key_phrases):
            # Find a reason why this attack is possible in the context
            reason = f"Possible because: {description}.\nScenario details may lead to {attack.lower()} due to the following reasons: "
            if "cloud-based systems" in scenario and attack == "Man-in-the-Middle":
                reason += "data transmission might be intercepted over insecure channels."
            elif "third-party APIs" in scenario and attack == "Phishing":
                reason += "attacks could target API access credentials."
            elif "remote work" in scenario and attack == "Ransomware":
                reason += "employees working remotely may download malicious files unknowingly."
            elif "personal devices" in scenario and attack == "Insider Threats":
                reason += "lack of device security may expose sensitive data to internal actors."
            elif "IoT devices" in scenario and attack == "DDoS":
                reason += "unsecured IoT devices could be exploited to flood the network."

            attack_results.append(f"Attack: {attack}\nReason: {reason}")

    # Format the results
    if not attack_results:
        return "No security concerns identified."
    
    return "\n\n".join(attack_results)

# Example usage
if __name__ == "__main__":
    # Provide a company scenario
    scenario = """
    A mid-sized company operates in the financial sector. They handle sensitive customer data, including personal 
    and financial information. Employees frequently access cloud-based systems for processing data and use 
    third-party APIs for transactions. Remote work is common, with employees using personal devices to log in. 
    The company also uses IoT devices for office automation.
    """

    # Define an attack map
    attack_map = {
        "Phishing": "A social engineering attack to steal sensitive information by impersonating a trustworthy entity.",
        "Ransomware": "Malware that encrypts a victim's files, demanding a ransom to restore access.",
        "Man-in-the-Middle": "An attack where a hacker intercepts communication between two parties to steal data.",
        "Insider Threats": "Security risks originating from within the organization, such as malicious employees.",
        "DDoS": "Distributed denial-of-service attacks to overwhelm systems, causing disruption of services."
    }

    # Ask the security-related question
    security_concerns = identify_security_concerns(scenario, attack_map)
    print(security_concerns)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Raw Response: Scenario: 
    A mid-sized company operates in the financial sector. They handle sensitive customer data, including personal 
    and financial information. Employees frequently access cloud-based systems for processing data and use 
    third-party APIs for transactions. Remote work is common, with employees using personal devices to log in. 
    The company also uses IoT devices for office automation.
    
Here is a map of potential security attacks and their descriptions:
Phishing: A social engineering attack to steal sensitive information by impersonating a trustworthy entity.
Ransomware: Malware that encrypts a victim's files, demanding a ransom to restore access.
Man-in-the-Middle: An attack where a hacker intercepts communication between two parties to steal data.
Insider Threats: Security risks originating from within the organization, such as malicious employees.
DDoS: Distributed denial-of-service attacks to overwhelm systems, causing disruption of services.
Que